# Text Processing

## load packages

In [1]:
#!pip install transformers 
#!pip install textblob 
#!pip install pyvis
#!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116


In [2]:
## Load Packages
import torch
from transformers import pipeline 
summarizer = pipeline ("summarization")

C:\Users\peter\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [3]:
# workmen 
# Import numpy and pandas to work with dataframes 
import numpy as np 
import pandas as pd 
import networkx as nx 
from pyvis.network import Network

# Import seaborn and matplotlib for viz 
from matplotlib import pyplot as plt

# Import stopwords 
#nltk.download('punkt')
#nltk.download ('averaged_perceptron_tagger') 
#nltk.download('brown')
import nltk 
from nltk.corpus import stopwords 
from nltk import punkt

# download stop words
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('omw-1.4')
#nltk.download('popular')
stop_words = stopwords.words('english')
custom_stopwords = ['', '']


# Import textblob 
from textblob import Word, TextBlob
from textblob.np_extractors import ConllExtractor




## Load text

In [4]:
# # Load text
raw_data = pd.read_csv('G:\\My Drive\\01. PlotSmith\\06. Tinkering\\Python\\trustpilot text NLP\\blueberry_text.csv') 

In [5]:
raw_data.head(5)

,date,title,detail
0,24/10/2023,sample 1,"Sure! Overall, my experience with Blueberry ha..."
1,24/10/2023,sample 2,"Well, to be honest, my overall experience with..."


In [6]:
#clean questionable characters 
raw_data = raw_data.replace(",", regex=True)
raw_data = raw_data.replace("!", regex=True)

# Cleaning

In [7]:
nltk.download('stopwords') 
nltk.download('wordnet') 
stop_words = stopwords.words ('english') 
custom_stopwords = [", "]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\peter\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\peter\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
def preprocess_revs(rev, custom_stopwords):
    processed_rev = rev 
    processed_rev.replace('[^\w\s]', '') 
    processed_rev = " ".join(word for word in processed_rev.split() if word not in stop_words) 
    processed_rev = " ".join(word for word in processed_rev.split() if word not in custom_stopwords) 
    processed_rev = " ".join(Word(word). lemmatize() for word in processed_rev.split()) 
    return(processed_rev)
                          
raw_data[ 'Processed rev'] = raw_data['detail'].apply(lambda x: preprocess_revs(x, custom_stopwords)) 
raw_data['Processed rev'] = raw_data['Processed rev'].apply(lambda x: x.lower()) 
raw_data.head()

,date,title,detail,Processed rev
0,24/10/2023,sample 1,"Sure! Overall, my experience with Blueberry ha...","sure! overall, experience blueberry quite posi..."
1,24/10/2023,sample 2,"Well, to be honest, my overall experience with...","well, honest, overall experience blueberry qui..."


## Summarize Text

In [9]:
raw_data['summary'] = raw_data['detail'].apply(lambda x: summarizer(x, max_length=50, min_length=15, do_sample=False)) 

In [10]:
for i in range(0,len(raw_data)):
    raw_data['summary'][i] =     raw_data['summary'][i][0]['summary_text']
    #print(i)

# Product mentions

In [11]:
blueberry_handle = ['blueberry', 'blueberry'] 
shipping_handle = ['shipping', 'shipping']

In [12]:
def identify_subject(rev, refs):
    flag = 0 
    for ref in refs: 
        if rev.find(ref) != -1:
            flag = 1 
    return flag


raw_data['blueberry'] = raw_data['Processed rev'].apply(lambda x: identify_subject(x, blueberry_handle)) 
raw_data['shipping'] = raw_data['Processed rev'].apply(lambda x: identify_subject(x, shipping_handle))

def notopic(blueberry, shipping): 
    if blueberry==1 or shipping==1:
        nither=0 
    else:
        nither=1 
    return nither

                               
raw_data['nether'] = raw_data.apply(lambda x: notopic(x['blueberry'], x['shipping']), axis=1) 
raw_data.head(10)

,date,title,detail,Processed rev,summary,blueberry,shipping,nether
0,24/10/2023,sample 1,"Sure! Overall, my experience with Blueberry ha...","sure! overall, experience blueberry quite posi...","Blueberry offers a wide range of products, co...",1,1,0
1,24/10/2023,sample 2,"Well, to be honest, my overall experience with...","well, honest, overall experience blueberry qui...",Blueberry's customer reviews seemed unreliabl...,1,0,0


## Sentiment and polarity

In [13]:
# Calculate polarity 
raw_data[ 'polarity'] = raw_data[ 'Processed rev'].apply(lambda x: TextBlob(x). sentiment[0]) 
raw_data['subjectivity'] = raw_data['Processed rev'].apply(lambda x: TextBlob(x). sentiment[1]) 
raw_data[['Processed rev', 'blueberry', 'shipping', 'nether', 'polarity', 'subjectivity']].head()

,Processed rev,blueberry,shipping,nether,polarity,subjectivity
0,"sure! overall, experience blueberry quite posi...",1,1,0,0.236185,0.488811
1,"well, honest, overall experience blueberry qui...",1,0,0,-0.048086,0.504279


In [14]:
display(raw_data[raw_data['blueberry' ]==1][['blueberry' , 'polarity', 'subjectivity']].groupby('blueberry' ).agg([np.mean, np.max, np.min, np.median])) 
display(raw_data[raw_data['shipping']==1][['shipping', 'polarity', 'subjectivity']].groupby('shipping').agg([np.mean, np.max, np.min, np.median])) 
display(raw_data[raw_data[ 'nether']==1][[ 'nether', 'polarity', 'subjectivity']].groupby('nether' ).agg([np.mean, np.max, np.min, np.median]))

polarity                              subjectivity            \
              mean       max       min   median         mean       max   
blueberry                                                                
1          0.09405  0.236185 -0.048086  0.09405     0.496545  0.504279   

                               
                min    median  
blueberry                      
1          0.488811  0.496545

polarity                               subjectivity            \
              mean       max       min    median         mean       max   
shipping                                                                  
1         0.236185  0.236185  0.236185  0.236185     0.488811  0.488811   

                              
               min    median  
shipping                      
1         0.488811  0.488811

Empty DataFrame
Columns: [(polarity, mean), (polarity, max), (polarity, min), (polarity, median), (subjectivity, mean), (subjectivity, max), (subjectivity, min), (subjectivity, median)]
Index: []

In [15]:
extractor = ConllExtractor()

In [16]:
raw_data['words'] = raw_data[ 'Processed rev'].apply(lambda x: TextBlob(x).words) 
raw_data[ 'tags'] = raw_data[ 'Processed rev'].apply(lambda x: TextBlob(x).tags)
raw_data['noun_phrase'] = raw_data[ 'Processed rev'].apply(lambda x: TextBlob(x, np_extractor=extractor).noun_phrases)


**********************************************************************
  Resource conll2000 not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('conll2000')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load corpora/conll2000

  Searched in:
    - 'C:\\Users\\peter/nltk_data'
    - 'C:\\Users\\peter\\anaconda3\\nltk_data'
    - 'C:\\Users\\peter\\anaconda3\\share\\nltk_data'
    - 'C:\\Users\\peter\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\peter\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************



MissingCorpusError: 
Looks like you are missing some required data for this feature.

To download the necessary data, simply run

    python -m textblob.download_corpora

or use the NLTK downloader to download the missing data: http://nltk.org/data.html
If this doesn't fix the problem, file an issue at https://github.com/sloria/TextBlob/issues.


In [ ]:
raw_data['index1'] = raw_data.index 
noun_phrase = raw_data[['index1', 'noun_phrase']]

In [ ]:
noun_phrase.explode('noun_phrase')

In [ ]:
#full_text =
for i in range(0, len(raw_data['Processed rev'])):
    raw_data[ 'Processed rev'][i]

In [ ]:
raw_data['bigrams'] = raw_data['Processed rev'].apply(lambda x: TextBlob(x).ngrams (n=2) )
raw_data['bigrams'].head()


In [ ]:
'first get individual words'
raw_data['tokens'] = raw_data['Processed rev'].apply(lambda x: TextBlob(x).split())


In [ ]:
bigrams = [] 
for doc in raw_data['tokens'] :
    bigrams.extend([(doc[i-1], doc[i])
        for i in range(1, len(doc))])

In [ ]:
bigrams_freq = [(b,bigrams.count(b))
               for b in set(bigrams)]

In [ ]:
bigrams_freq_df = pd.DataFrame(bigrams_freq, columns = ['SourceTarget', 'Weight'])

In [ ]:
bigrams_freq_df['Source'] = bigrams_freq_df[ 'SourceTarget'].apply(lambda x: x[0]) 
bigrams_freq_df['Target'] = bigrams_freq_df[ 'SourceTarget'].apply(lambda x: x[1])

In [ ]:
bigrams_freq_df['Type'] = 'Undirected'
bigrams_freq_df[['Source', 'Target', 'Type', 'Weight']]
net_data = bigrams_freq_df[bigrams_freq_df['Weight'] >= 2]

In [ ]:
net_data.dtypes

In [ ]:

G = nx.from_pandas_edgelist(net_data, source = 'Source', target = 'Target', edge_attr='Weight')

net = Network(notebook = True) 
net.from_nx(G) 
net.show("example.html")
